In [1]:
# # 도시명 집계함수 사ㅏ용 불가능 -> 오브젝트 제거 
# # 그룹하는데 기준할땐 사용할수있음 
# df_near = df_near.drop('도시명',axis=1).groupby(['군집','일시']).mean().reset_index()
# # 군집과 일시로 묶음 
# # 군집별일시
# 멀라시팔

# from functools import reduce
# import pandas as pd
# dfs = [df_coldwave, df_kr, df_near]
# cold_kr_near = reduce(lambda left, right: pd.merge(left,right,left_index=True, right_index=True), dfs)

# 일시.value_counts() 한다음에 일시는 무조건 1개만 있어야함..
# 일시가 여러개일경우 오브젝트 칼럼 의심
# 일시와 groupby사용해서 오브젝트컬럼 묶어줌...
# 그다음 여러개의 값의 개수가있는 오브젝트 컬럼을 옆으로 붙여줌. 
# 컬럼을 옆으로 붙인 데이터 저장. 
# 머신러닝을 위한 데이터들 따로저장
# 재난과 기후 
# 피쳐임포턴스를 시각화 하여 중요한 피쳐만 뽑아서 모델돌려보기 . 
# 한파- 주변국/한파- 한국/한파- 해상/한파- 북한 .. 근데 북한은 한국과 합치는게 낫지않을까 . 
# 한파 -한국,주변국 /한파 - 한국,북한/ 한파 -한국, 해상 

In [2]:
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import os
from functools import reduce
import warnings 
warnings.filterwarnings('ignore')

In [3]:
# 분류모델 import 
# Logistic Regression,SGD,KNN,SVM,Decision Tree, Random Forest, LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report,roc_auc_score,f1_score

lr = LogisticRegression()
knn = KNeighborsClassifier()
svm = SVC()
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier()
lgbm = LGBMClassifier()

In [120]:
df_coldwave = pd.read_csv('./data/preprocessed/df_coldwave.csv',encoding='cp949',index_col=0)
df_kr = pd.read_csv('./data/preprocessed/국내기후(백령도제외).csv',index_col=0)
df_near = pd.read_csv('./data/preprocessed/preprocessed_주변국 기상데이터_199601to202108.csv',index_col=0)
df_sea = pd.read_csv('./data/preprocessed/국내_바다.csv',index_col=0)
df_high = pd.read_csv('./data/preprocessed/preprocessed_고층기후데이터.csv',index_col=0)
df_heatwave = pd.read_csv('./data/preprocessed/df_heatwave.csv',encoding='cp949',index_col=0)
df_tropical = pd.read_csv('./data/preprocessed/df_troplical.csv',encoding='cp949',index_col=0)

In [5]:
df_coldwave
# 날짜 중복(지점때문에)
# 널값은 없음 
# 일시와 재난일어난날 object 

,연도,날짜,지점,일시,재난일어난날
0,1973,103,이천,1973-01-03,1973-01-03
1,1973,103,수원,1973-01-03,1973-01-03
2,1973,103,임실,1973-01-03,1973-01-03
3,1973,103,서산,1973-01-03,1973-01-03
4,1973,103,대관령,1973-01-03,1973-01-03
...,...,...,...,...,...
21982,2022,224,장수,2022-02-24,2022-02-24
21983,2022,224,대관령,2022-02-24,2022-02-24
21984,2022,224,인제,2022-02-24,2022-02-24
21985,2022,224,이천,2022-02-24,2022-02-24


In [6]:
df_kr
# 시간과 관측지점 object
# 시간이름 일시 
# 널값없음 

,시간,관측지점,평균 기온,최저 기온,최고 기온,1시간 최다강수량,일강수량,최대 순간풍속,최대 순간 풍속 풍향,최대 풍속,...,합계 일사,일 최심신적설,일 최심적설,합계 3시간 신적설,평균 전운량,평균 중하층운량,평균 지면온도,합계 대형증발량,합계 소형증발량,안개 계속 시간
0,1990-01-01,서울,-1.7,-4.0,1.8,0.0,0.0,7.3,270.0,5.0,...,4.58,0.0,0.0,0.0,3.5,3.6,-1.5,2.721902,0.9,0.0
1,1990-01-02,서울,-3.5,-5.7,-0.9,0.0,0.1,8.6,290.0,4.8,...,3.44,0.1,0.1,0.1,3.5,3.8,-2.9,2.721902,0.8,0.0
2,1990-01-03,서울,-5.0,-7.2,-1.7,0.0,0.0,8.2,290.0,5.0,...,7.63,0.0,0.0,0.0,0.0,0.0,-4.4,2.721902,1.4,0.0
3,1990-01-04,서울,-3.3,-8.9,2.1,0.0,0.0,6.1,290.0,3.0,...,5.84,0.0,0.0,0.0,0.5,1.8,-3.6,2.721902,1.0,0.0
4,1990-01-05,서울,0.7,-2.4,2.1,0.0,3.2,12.3,290.0,7.2,...,0.36,0.6,0.6,0.6,9.0,3.6,-0.6,2.721902,1.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11683,2021-12-27,제주,4.1,0.8,6.5,0.0,0.1,12.8,20.0,6.8,...,1.37,0.5,1.2,0.5,8.1,8.1,3.9,1.000000,1.4,0.0
11684,2021-12-28,제주,7.2,5.9,8.4,0.0,0.0,9.1,360.0,5.3,...,0.91,0.0,0.0,0.0,8.3,8.0,5.6,0.900000,1.3,0.0
11685,2021-12-29,제주,9.3,6.6,11.5,0.0,0.4,12.8,340.0,7.9,...,4.94,0.0,0.0,0.0,8.5,7.8,6.9,2.300000,3.3,0.0
11686,2021-12-30,제주,6.9,5.8,8.4,0.0,0.0,18.1,340.0,10.1,...,4.85,0.0,0.0,0.0,8.0,8.0,5.9,2.300000,3.2,0.0


In [7]:
df_near.isnull().sum()
# 인덱스가 9370인데 rows가 271875가 나왔으면 중복값이 있다는건데 이걸 어케아냐면 의심가는 칼럼의 value_counts()해보면 각 밸류가 몇개나있는지알수잇음
# 여튼 지역별 날짜 중복됨 예를들어 도쿄도 1월1일 창진도 1월1일 이런식으로 
# 군집 도시명 일시 object
# 널값없음 
# 저 중복값을 없애기위해 군집과 날짜를 groupby함. 군집별 날짜 이렇게 

도시명       0
일시        0
평균기온      0
최고기온      0
최저기온      0
습도        0
풍향        0
풍속        0
강수량       0
해면기압      0
이슬점 온도    0
군집        0
dtype: int64

In [8]:
# df_coldwave 전처리 

In [9]:
# 중복값제거할수없게만드는 column 삭제
df_coldwave.drop('지점',axis=1,inplace=True)

In [10]:
# 중복값제거함으로써 똑같은날짜 없앰
df_coldwave.drop_duplicates(inplace=True)

In [11]:
df_coldwave.drop(['연도','날짜'],axis=1,inplace=True)
#연도날짜칼럼 삭제

In [12]:
df_coldwave.to_csv('한파.csv',encoding='cp949')

In [13]:
# 한국기후 전처리

In [14]:
df_kr
# 시간과 관측지점 object
# 시간이름 일시 
# 널값없음 

,시간,관측지점,평균 기온,최저 기온,최고 기온,1시간 최다강수량,일강수량,최대 순간풍속,최대 순간 풍속 풍향,최대 풍속,...,합계 일사,일 최심신적설,일 최심적설,합계 3시간 신적설,평균 전운량,평균 중하층운량,평균 지면온도,합계 대형증발량,합계 소형증발량,안개 계속 시간
0,1990-01-01,서울,-1.7,-4.0,1.8,0.0,0.0,7.3,270.0,5.0,...,4.58,0.0,0.0,0.0,3.5,3.6,-1.5,2.721902,0.9,0.0
1,1990-01-02,서울,-3.5,-5.7,-0.9,0.0,0.1,8.6,290.0,4.8,...,3.44,0.1,0.1,0.1,3.5,3.8,-2.9,2.721902,0.8,0.0
2,1990-01-03,서울,-5.0,-7.2,-1.7,0.0,0.0,8.2,290.0,5.0,...,7.63,0.0,0.0,0.0,0.0,0.0,-4.4,2.721902,1.4,0.0
3,1990-01-04,서울,-3.3,-8.9,2.1,0.0,0.0,6.1,290.0,3.0,...,5.84,0.0,0.0,0.0,0.5,1.8,-3.6,2.721902,1.0,0.0
4,1990-01-05,서울,0.7,-2.4,2.1,0.0,3.2,12.3,290.0,7.2,...,0.36,0.6,0.6,0.6,9.0,3.6,-0.6,2.721902,1.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11683,2021-12-27,제주,4.1,0.8,6.5,0.0,0.1,12.8,20.0,6.8,...,1.37,0.5,1.2,0.5,8.1,8.1,3.9,1.000000,1.4,0.0
11684,2021-12-28,제주,7.2,5.9,8.4,0.0,0.0,9.1,360.0,5.3,...,0.91,0.0,0.0,0.0,8.3,8.0,5.6,0.900000,1.3,0.0
11685,2021-12-29,제주,9.3,6.6,11.5,0.0,0.4,12.8,340.0,7.9,...,4.94,0.0,0.0,0.0,8.5,7.8,6.9,2.300000,3.3,0.0
11686,2021-12-30,제주,6.9,5.8,8.4,0.0,0.0,18.1,340.0,10.1,...,4.85,0.0,0.0,0.0,8.0,8.0,5.9,2.300000,3.2,0.0


In [15]:
# 시간컬럼을 일시로 이름바꾸기 
df_kr.rename(columns={'시간':'일시'},inplace=True)

In [16]:
df_kr.columns

Index(['일시', '관측지점', '평균 기온', '최저 기온', '최고 기온', '1시간 최다강수량', '일강수량', '최대 순간풍속',
       '최대 순간 풍속 풍향', '최대 풍속', '최대 풍속 풍향', '평균 풍속', '풍정합', '최대 풍향', '평균 이슬점온도',
       '평균 상대습도', '평균 증기압', '평균 현지기압', '최고 해면 기압', '최저 해면기압', '평균 해면기압',
       '합계 일조 시간', '1시간 최다 일사량', '합계 일사', '일 최심신적설', '일 최심적설', '합계 3시간 신적설',
       '평균 전운량', '평균 중하층운량', '평균 지면온도', '합계 대형증발량', '합계 소형증발량', '안개 계속 시간'],
      dtype='object')

In [17]:
tdf = pd.DataFrame({'일시':df_kr['일시'].unique()})
gunjip = list(df_kr['관측지점'].unique())
for hpa in gunjip:
    tmp = df_kr[df_kr['관측지점']==hpa].drop('관측지점',axis=1)
    tmp.columns = ['일시', f'{hpa}평균기온', f'{hpa}최저기온', f'{hpa}최고기온', f'{hpa}1시간 최다강수량', f'{hpa}일강수량', f'{hpa}최대 순간풍속', f'{hpa}최대 순간 풍속 풍향', f'{hpa}최대 풍속',
                   f'{hpa}최대 풍속 풍향',f'{hpa}평균 풍속',f'{hpa}풍정합',f'{hpa}최대 풍향',f'{hpa}평균 이슬점온도',f'{hpa}평균 상대습도'
                  ,f'{hpa}평균 증기압',f'{hpa}평균 현지기압',f'{hpa}최고 해면 기압',f'{hpa}최저 해면기압',f'{hpa}평균 해면기압',f'{hpa}합계 일조 시간',f'{hpa}1시간 최다 일사량',f'{hpa}합계 일사',
                   f'{hpa}일 최심신적설',f'{hpa}일 최심적설',f'{hpa}합계 3시간 신적설',f'{hpa}평균 전운량',f'{hpa}평균 중하층운량',f'{hpa}평균 지면온도',f'{hpa}합계 대형증발량',f'{hpa}합계 소형증발량',f'{hpa}안개 계속 시간']
    tdf = pd.merge(tdf,tmp, how='left',on='일시')
tdf

,일시,서울평균기온,서울최저기온,서울최고기온,서울1시간 최다강수량,서울일강수량,서울최대 순간풍속,서울최대 순간 풍속 풍향,서울최대 풍속,서울최대 풍속 풍향,...,제주합계 일사,제주일 최심신적설,제주일 최심적설,제주합계 3시간 신적설,제주평균 전운량,제주평균 중하층운량,제주평균 지면온도,제주합계 대형증발량,제주합계 소형증발량,제주안개 계속 시간
0,1990-01-01,-1.7,-4.0,1.8,0.0,0.0,7.3,270.0,5.0,270.0,...,5.52,0.0,0.0,0.0,5.3,5.8,4.6,1.0,1.4,0.0
1,1990-01-02,-3.5,-5.7,-0.9,0.0,0.1,8.6,290.0,4.8,290.0,...,2.65,0.0,0.0,0.0,8.8,8.1,4.6,1.0,1.6,0.0
2,1990-01-03,-5.0,-7.2,-1.7,0.0,0.0,8.2,290.0,5.0,290.0,...,2.40,0.0,0.0,0.0,9.3,9.3,3.2,1.0,1.6,0.0
3,1990-01-04,-3.3,-8.9,2.1,0.0,0.0,6.1,290.0,3.0,290.0,...,2.89,0.0,0.0,0.0,8.0,8.4,3.5,0.8,1.1,0.0
4,1990-01-05,0.7,-2.4,2.1,0.0,3.2,12.3,290.0,7.2,290.0,...,4.64,0.0,0.0,0.0,7.5,5.8,6.5,1.0,1.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11683,2021-12-27,-7.6,-12.9,-3.9,0.0,0.0,5.5,340.0,3.1,20.0,...,1.37,0.5,1.2,0.5,8.1,8.1,3.9,1.0,1.4,0.0
11684,2021-12-28,-4.1,-8.5,-0.9,0.0,0.0,4.9,50.0,3.1,50.0,...,0.91,0.0,0.0,0.0,8.3,8.0,5.6,0.9,1.3,0.0
11685,2021-12-29,0.4,-3.8,5.9,0.0,0.2,10.8,270.0,5.9,270.0,...,4.94,0.0,0.0,0.0,8.5,7.8,6.9,2.3,3.3,0.0
11686,2021-12-30,-3.9,-6.8,0.2,0.0,0.0,12.3,320.0,6.6,290.0,...,4.85,0.0,0.0,0.0,8.0,8.0,5.9,2.3,3.2,0.0


In [18]:
# 주변국 전처리

In [19]:
df_near.drop('도시명',axis=1,inplace=True)

In [24]:
df_near = df_near.groupby(['군집','일시']).mean()
df_near.reset_index(inplace=True)

In [22]:
tdf = pd.DataFrame({'일시':df_near['일시'].unique()})

In [25]:
gunjip = list(df_near['군집'].unique())
for hpa in gunjip:
    tmp = df_near[df_near['군집']==hpa].drop('군집',axis=1)
    tmp.columns = ['일시', f'{hpa}일시', f'{hpa}평균기온', f'{hpa}최고기온', f'{hpa}습도', f'{hpa}풍향', f'{hpa}풍속', f'{hpa}강수량', f'{hpa}해면기압', f'{hpa}이슬점 온도']
    tdf = pd.merge(tdf,tmp, how='left',on='일시')
tdf

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\sim\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\sim\AppData\Local\Temp/ipykernel_3896/559920562.py", line 5, in <module>
    tdf = pd.merge(tdf,tmp, how='left',on='일시')
  File "C:\Users\sim\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py", line 122, in merge
    return op.get_result()
  File "C:\Users\sim\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py", line 725, in get_result
    result_data = concatenate_managers(
  File "C:\Users\sim\Anaconda3\lib\site-packages\pandas\core\internals\concat.py", line 202, in concatenate_managers
    return _concat_managers_axis0(mgrs_indexers, axes, copy)
  File "C:\Users\sim\Anaconda3\lib\site-packages\pandas\core\internals\concat.py", line 264, in _concat_managers_axis0
    mgrs_indexers = _maybe_reindex_columns_na_proxy(axes, mgrs_indexers)
  File "C:\Users\sim\Anaco

TypeError: object of type 'NoneType' has no len()

In [ ]:
# tdf.to_csv('주변국기후.csv',encoding='cp949')

In [ ]:
# 해상기후 전처리

In [70]:
df_sea['시간'].value_counts()

2001-01-01    3
2008-01-07    3
2007-12-26    3
2007-12-27    3
2007-12-28    3
             ..
1993-09-27    2
1993-09-28    2
1993-09-29    2
1993-09-30    2
1980-01-01    2
Name: 시간, Length: 15342, dtype: int64

In [105]:
df_sea = df_sea.groupby(['관측지점','시간']).mean()

In [109]:
df_sea.rename(columns={'시간':'일시'},inplace=True)

In [107]:
df_sea.reset_index(inplace=True)

In [110]:
df_sea.columns

Index(['관측지점', '일시', '평균 기온', '최저 기온', '최고 기온', '1시간 최다강수량', '일강수량', '최대 순간풍속',
       '최대 순간 풍속 풍향', '최대 풍속', '최대 풍속 풍향', '평균 풍속', '풍정합', '최대 풍향', '평균 이슬점온도',
       '평균 상대습도', '평균 증기압', '평균 현지기압', '최고 해면 기압', '최저 해면기압', '평균 해면기압',
       '합계 일조 시간', '1시간 최다 일사량', '합계 일사', '일 최심신적설', '일 최심적설', '합계 3시간 신적설',
       '평균 전운량', '평균 중하층운량', '평균 지면온도', '합계 대형증발량', '합계 소형증발량', '안개 계속 시간'],
      dtype='object')

In [111]:
tdf = pd.DataFrame({'일시':df_sea['일시'].unique()})
gunjip = list(df_sea['관측지점'].unique())
for hpa in gunjip:
    tmp = df_sea[df_sea['관측지점']==hpa].drop('관측지점',axis=1)
    tmp.columns = ['일시', f'{hpa}평균 기온', f'{hpa}최저 기온', f'{hpa}최고 기온', f'{hpa}1시간 최다강수량', f'{hpa}일강수량', f'{hpa}최대 순간풍속', f'{hpa}최대 순간 풍속 풍향', f'{hpa}최대 풍속',
                  f'{hpa}최대 풍속 풍향', f'{hpa}평균 풍속', f'{hpa}풍정합', f'{hpa}최대 풍향', f'{hpa}평균 이슬점온도', f'{hpa}평균 상대습도', f'{hpa}평균 증기압', f'{hpa}평균 현지기압', f'{hpa}최고 해면 기압',
                  f'{hpa}최저 해면기압', f'{hpa}평균 해면기압', f'{hpa}합계 일조 시간', f'{hpa}1시간 최다 일사량', f'{hpa}합계 일사', f'{hpa}일 최심신적설', f'{hpa}일 최심적설', f'{hpa}합계 3시간 신적설', f'{hpa}평균 전운량',
                  f'{hpa}평균 중하층운량', f'{hpa}평균 지면온도', f'{hpa}합계 대형증발량', f'{hpa}합계 소형증발량', f'{hpa}안개 계속 시간']
    tdf = pd.merge(tdf,tmp, how='right',on='일시')
tdf

,일시,백령도평균 기온,백령도최저 기온,백령도최고 기온,백령도1시간 최다강수량,백령도일강수량,백령도최대 순간풍속,백령도최대 순간 풍속 풍향,백령도최대 풍속,백령도최대 풍속 풍향,...,제주합계 일사,제주일 최심신적설,제주일 최심적설,제주합계 3시간 신적설,제주평균 전운량,제주평균 중하층운량,제주평균 지면온도,제주합계 대형증발량,제주합계 소형증발량,제주안개 계속 시간
0,1980-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.900115,0.0,0.0,0.0,10.0,-999.0,10.7,2.614596,1.5,0.0
1,1980-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.900115,0.0,0.0,0.0,10.0,-999.0,12.4,2.614596,1.0,0.0
2,1980-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.900115,0.0,0.0,0.0,10.0,-999.0,9.1,2.614596,0.8,0.0
3,1980-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.900115,0.0,0.0,0.0,9.8,-999.0,4.7,2.614596,2.7,0.0
4,1980-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.900115,0.0,0.0,0.0,10.0,-999.0,2.8,2.614596,1.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15337,2021-12-28,2.0,-1.4,5.8,0.0,0.0,9.5,140.0,7.2,140.0,...,0.910000,0.0,0.0,0.0,8.3,8.0,5.6,0.900000,1.3,0.0
15338,2021-12-29,2.7,-1.7,6.5,0.0,0.0,19.8,340.0,13.3,340.0,...,4.940000,0.0,0.0,0.0,8.5,7.8,6.9,2.300000,3.3,0.0
15339,2021-12-30,-0.8,-2.2,1.4,0.0,0.3,21.1,340.0,14.2,340.0,...,4.850000,0.0,0.0,0.0,8.0,8.0,5.9,2.300000,3.2,0.0
15340,2021-12-31,-3.1,-3.9,-1.5,0.0,0.0,20.6,340.0,13.4,340.0,...,2.300000,0.0,0.0,0.0,8.5,8.5,4.9,1.300000,1.8,0.0


In [112]:
li = ['백령도']
for hpa in li:
    tdf.drop([ f'{hpa}평균 기온', f'{hpa}최저 기온', f'{hpa}최고 기온', f'{hpa}1시간 최다강수량', f'{hpa}일강수량', f'{hpa}최대 순간풍속', f'{hpa}최대 순간 풍속 풍향', f'{hpa}최대 풍속',
                  f'{hpa}최대 풍속 풍향', f'{hpa}평균 풍속', f'{hpa}풍정합', f'{hpa}최대 풍향', f'{hpa}평균 이슬점온도', f'{hpa}평균 상대습도', f'{hpa}평균 증기압', f'{hpa}평균 현지기압', f'{hpa}최고 해면 기압',
                  f'{hpa}최저 해면기압', f'{hpa}평균 해면기압', f'{hpa}합계 일조 시간', f'{hpa}1시간 최다 일사량', f'{hpa}합계 일사', f'{hpa}일 최심신적설', f'{hpa}일 최심적설', f'{hpa}합계 3시간 신적설', f'{hpa}평균 전운량',
                  f'{hpa}평균 중하층운량', f'{hpa}평균 지면온도', f'{hpa}합계 대형증발량', f'{hpa}합계 소형증발량', f'{hpa}안개 계속 시간'],axis=1,inplace=True)

In [115]:
# df_sea.set_index('관측지점').loc['울릉도'].sort_values(by='일시',ascending=True)


# #백령도 2000년부터 시작
# #제주 1980년 부터 시작
# #울릉도 1980년부터 시작
tdf.to_csv('해상기후.csv',encoding='cp949')

In [90]:
tdf.to_csv('해상기후.csv',encoding='cp949')

In [ ]:
# 고층기후 전처리 

In [102]:
df_high['일시'].value_counts().sort_values(ascending=False)

1990-01-01    1
1990-02-13    1
1990-01-23    1
1990-03-31    1
1990-01-22    1
             ..
2021-12-25    1
2021-12-26    1
2021-12-27    1
2021-12-28    1
2021-12-31    1
Name: 일시, Length: 11688, dtype: int64

In [117]:
df_high.to_csv('고층기후.csv',encoding='cp949')

In [ ]:
# 열대야랑 폭염 전처리

In [123]:
df_heatwave.drop(['연도','날짜','지점'],axis=1,inplace=True)

In [125]:
df_heatwave.drop_duplicates(inplace=True)

In [135]:
df_heatwave.to_csv('폭염.csv',encoding='cp949')

In [128]:
df_tropical
df_tropical.drop(['연도','날짜','지점'],axis=1,inplace=True)

In [131]:
df_tropical.reset_index(inplace=True)

In [134]:
df_tropical.drop_duplicates(inplace=True)

In [136]:
df_tropical.to_csv('열대야.csv',encoding='cp949')

In [137]:
# 2019년 데이터 제거하기 

In [152]:
열대야 = pd.read_csv('./data/ML_preprocessed/열대야.csv',encoding='cp949',index_col=0)
고층기후 = pd.read_csv('./data/ML_preprocessed/고층기후.csv',encoding='cp949',index_col=0)
주변국기후 = pd.read_csv('./data/ML_preprocessed/주변국기후.csv',encoding='cp949',index_col=0)
폭염 = pd.read_csv('./data/ML_preprocessed/폭염.csv',encoding='cp949',index_col=0)
한국기후 = pd.read_csv('./data/ML_preprocessed/한국기후.csv',encoding='cp949',index_col=0)
한파 = pd.read_csv('./data/ML_preprocessed/한파.csv',encoding='cp949',index_col=0)
해상기후 = pd.read_csv('./data/ML_preprocessed/해상기후.csv',encoding='cp949',index_col=0)

In [163]:
주변국기후

,일시,n1일시,n1평균기온,n1최고기온,n1습도,n1풍향,n1풍속,n1강수량,n1해면기압,n1이슬점 온도,...,n6이슬점 온도,n7일시,n7평균기온,n7최고기온,n7습도,n7풍향,n7풍속,n7강수량,n7해면기압,n7이슬점 온도
0,1996-01-01,-3.639583,0.5000,-8.0875,48.419643,218.750000,1.169643,0.000000,1022.948810,-13.566369,...,-6.596667,3.833333,4.566667,3.133333,69.777778,123.333333,2.555556,0.000000,1019.444444,-2.877778
1,1996-01-02,0.900000,3.7500,-2.8125,71.203125,165.000000,2.515625,0.446631,1017.043750,-3.859375,...,-18.627143,6.112500,13.100000,-1.400000,44.841667,206.666667,3.625000,0.000000,1023.658333,-6.316667
2,1996-01-03,-6.095313,0.0750,-8.7750,52.091518,253.750000,4.433265,0.446631,1021.464062,-14.331250,...,-21.976429,1.954167,5.333333,-0.766667,53.358333,60.000000,3.000000,0.000000,1028.558333,-7.183333
3,1996-01-04,-7.325000,-1.9750,-12.4375,59.312500,198.348983,1.484375,0.445600,1025.557812,-14.362500,...,-12.009286,5.637500,10.966667,0.366667,64.408333,240.000000,2.726190,0.000000,1019.637500,-0.950000
4,1996-01-05,-1.073214,1.1750,-3.0500,80.558036,230.524917,1.966518,1.623677,1021.357143,-4.158482,...,-12.091071,6.191667,10.866667,2.066667,67.700000,210.000000,2.375000,0.000000,1021.525000,0.533333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9370,2021-08-27,23.270312,24.9500,21.7125,82.468750,111.250000,0.906250,0.880792,1011.759375,20.028125,...,16.523214,25.229167,28.333333,22.500000,78.830556,240.000000,2.125000,-0.047822,1012.870833,21.366667
9371,2021-08-28,24.135938,28.5375,20.8125,77.921875,115.000000,1.500000,0.000000,1013.007812,19.660937,...,16.480000,26.490476,30.100000,23.133333,81.301190,106.666667,1.404762,-0.739609,1012.662500,22.993452
9372,2021-08-29,23.090625,26.2000,20.8500,83.125000,162.500000,1.031250,1.669771,1013.484375,19.915625,...,17.045000,25.787500,29.033333,23.233333,87.795833,210.000000,2.083333,-0.702848,1013.058333,23.450000
9373,2021-08-30,23.782812,28.3375,20.4375,77.578125,183.750000,0.937500,0.000000,1015.429688,19.173437,...,17.566429,27.412500,30.700000,25.233333,83.704167,220.000000,1.916667,-0.625120,1012.554167,24.179167


In [164]:
# 열대야.set_index('일시',inplace=True)
주변국기후.set_index('일시',inplace=True)
폭염.set_index('일시',inplace=True)
한국기후.set_index('일시',inplace=True)
한파.set_index('일시',inplace=True)
해상기후.set_index('일시',inplace=True)

In [171]:
열대야

,재난일어난날
일시,
1973-07-06,1973-07-06
1973-07-07,1973-07-07
1973-07-08,1973-07-08
1973-07-09,1973-07-09
1973-07-10,1973-07-10
...,...
2021-08-27,2021-08-27
2021-08-28,2021-08-28
2021-08-29,2021-08-29


In [169]:
열대야2019 = 열대야.loc['2019-01-01':'2019-12-31']
한파2019 = 한파.loc['2019-01-01':'2019-12-31']
폭염2019 = 폭염.loc['2019-01-01':'2019-12-31']
주변국기후2019 = 주변국기후.loc['2019-01-01':'2019-12-31']
한국기후2019 = 한국기후.loc['2019-01-01':'2019-12-31']
해상기후2019 = 해상기후.loc['2019-01-01':'2019-12-31']
고층기후2019 = 고층기후.loc['2019-01-01':'2019-12-31']

Index(['2019-05-24', '2019-05-25', '2019-07-19', '2019-07-21', '2019-07-22',
       '2019-07-23', '2019-07-24', '2019-07-25', '2019-07-26', '2019-07-27',
       '2019-07-28', '2019-07-29', '2019-07-30', '2019-07-31', '2019-08-01',
       '2019-08-02', '2019-08-03', '2019-08-04', '2019-08-05', '2019-08-06',
       '2019-08-07', '2019-08-08', '2019-08-09', '2019-08-10', '2019-08-11',
       '2019-08-12', '2019-08-13', '2019-08-14', '2019-08-15', '2019-08-16',
       '2019-08-17', '2019-08-18', '2019-08-19', '2019-08-20', '2019-08-21',
       '2019-08-22', '2019-09-05', '2019-09-06', '2019-09-07', '2019-09-08',
       '2019-09-09', '2019-09-10'],
      dtype='object', name='일시')

In [184]:
열대야2019drop = 열대야.drop(열대야2019.index,axis=0)
한파2019drop = 한파.drop(한파2019.index,axis=0)
폭염2019drop = 폭염.drop(폭염2019.index,axis=0)
주변국기후2019drop = 주변국기후.drop(주변국기후2019.index,axis=0)
한국기후2019drop = 한국기후.drop(한국기후2019.index,axis=0)
해상기후2019drop = 해상기후.drop(해상기후2019.index,axis=0)
고층기후2019drop = 고층기후.drop(고층기후2019.index,axis=0)